<a href="https://colab.research.google.com/github/Alfonso-Jesus-Garcia-Moya/Tecnicas_de_inteligencia_artificial/blob/main/SEM%2012%20TAREA%20DE%20Algoritmos%20gen%C3%A9ticos%3A%20encuentra%20la%20frase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Con estas líneas se crea un archivo con extensión .py en modo escritura
with open("ga_string_evolver.py", "w") as f:
    f.write("""
import random
import string
from typing import List, Tuple

# Se define el modo del alfabeto: 'target' usa solo caracteres que ya existen en la frase objetivo
ALPHABET_MODE = 'target'
FULL_ALPHABET = string.ascii_letters + string.punctuation + string.digits + " "

def build_alphabet(target: str) -> str:
    if ALPHABET_MODE == 'full':
        return FULL_ALPHABET
    # Se obtienen caracteres únicos del target ordenados
    s = sorted(set(target + " "))
    return "".join(s)

Individual = List[str]

def random_char_from(alpha: str) -> str:
    return random.choice(alpha)

def random_individual(n: int, alpha: str) -> Individual:
    return [random_char_from(alpha) for _ in range(n)]

def fitness(ind: Individual, target: str) -> int:
    # Se suma 1 por cada coincidencia exacta
    return sum(1 for g, t in zip(ind, target) if g == t)

def tournament_select(pop: List[Individual], fits: List[int], k: int = 3) -> Individual:
    # Se eligen k índices aleatorios
    contestants = random.sample(range(len(pop)), k)
    # Se selecciona el de mejor fitness
    best_i = max(contestants, key=lambda i: fits[i])
    return pop[best_i][:]

def crossover(p1: Individual, p2: Individual, *, uniform: bool = False) -> Tuple[Individual, Individual]:
    if len(p1) == 1:
        return p1[:], p2[:]

    if uniform:
        c1, c2 = p1[:], p2[:]
        for i in range(len(p1)):
            if random.random() < 0.5:
                c1[i], c2[i] = c2[i], c1[i]
        return c1, c2

    # Cruce de un punto
    cut = random.randint(1, len(p1)-1)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def mutate(ind: Individual, mutation_rate: float, alpha: str) -> None:
    if len(ind) == 0:
        return
    p = mutation_rate / len(ind)
    for i in range(len(ind)):
        if random.random() < p:
            ind[i] = random_char_from(alpha)

def best_of(pop: List[Individual], target: str) -> Tuple[Individual, int]:
    best = pop[0]
    best_fit = fitness(best, target)
    for ind in pop[1:]:
        f = fitness(ind, target)
        if f > best_fit:
            best = ind
            best_fit = f
    return best, best_fit

def pretty(ind: Individual) -> str:
    return "".join(ind)

def evolve_string(
    target: str = "Hola, algoritmos geneticos!",
    population_size: int = 200,
    generations: int = 1000,
    crossover_rate: float = 0.9,
    mutation_rate: float = 2.0,
    tournament_k: int = 3,
    seed: int = None,
    verbose_every: int = 1,
    uniform_crossover: bool = True,
    patience: int = 50,
    mut_increase: float = 1.5,
    mut_cap: float = 6.0
):
    if seed is not None:
        random.seed(seed)

    alpha = build_alphabet(target)
    L = len(target)
    pop = [random_individual(L, alpha) for _ in range(population_size)]

    best, best_fit = best_of(pop, target)
    if verbose_every:
        print(f"Generation: 0 String: {pretty(best)} Fitness: {best_fit}")

    prev_best = -1
    no_improve = 0

    for gen in range(1, generations + 1):
        fits = [fitness(ind, target) for ind in pop]

        # Elitismo
        elite_idx = max(range(len(pop)), key=lambda i: fits[i])
        elite = pop[elite_idx][:]
        elite_fit = fits[elite_idx]

        new_pop = [elite]

        while len(new_pop) < population_size:
            p1 = tournament_select(pop, fits, tournament_k)
            p2 = tournament_select(pop, fits, tournament_k)

            if random.random() < crossover_rate:
                c1, c2 = crossover(p1, p2, uniform=uniform_crossover)
            else:
                c1, c2 = p1[:], p2[:]

            mutate(c1, mutation_rate, alpha)
            if len(new_pop) < population_size:
                new_pop.append(c1)

            mutate(c2, mutation_rate, alpha)
            if len(new_pop) < population_size:
                new_pop.append(c2)

        pop = new_pop
        best, best_fit = best_of(pop, target)

        if verbose_every and (gen % verbose_every == 0):
            print(f"Generation: {gen} String: {pretty(best)} Fitness: {best_fit}")

        if best_fit == L:
            return pretty(best), best_fit, gen

        # Adaptación de la tasa de mutación
        if best_fit > prev_best:
            mutation_rate = max(2.0, mutation_rate / (mut_increase ** 0.5))
            no_improve = 0
            prev_best = best_fit
        else:
            no_improve += 1
            if no_improve >= patience:
                mutation_rate = min(mut_cap, mutation_rate * mut_increase)
                no_improve = 0

    return pretty(best), best_fit, generations
""")

In [3]:
from ga_string_evolver import evolve_string

# Ejecutamos el algoritmo
s, f, g = evolve_string(
    target="Hola, algoritmos geneticos",
    population_size=600,     # Aumentamos población (PDF original usaba 300)
    generations=200,
    crossover_rate=0.9,
    mutation_rate=2.0,
    tournament_k=10,         # Aumentamos presión (PDF original usaba 5)
    uniform_crossover=True,
    patience=40,
    mut_increase=1.7,
    mut_cap=8.0,
    seed=None,               # Quitamos la semilla 7 para buscar un resultado distinto
    verbose_every=1
)

print(f"\nResultado final: {s} | Fitness: {f} | Generación: {g}")

Generation: 0 String: tilligHa Heirmnmmeeaeorcoe Fitness: 7
Generation: 1 String: HocaaeiceH,iHmnrngsnlnc rs Fitness: 8
Generation: 2 String: ,oll, ialcetcsotm,eaeorcos Fitness: 10
Generation: 3 String: trla,cHagHmHtmgilgeaelicos Fitness: 13
Generation: 4 String: H rac alorritmaamgggetioos Fitness: 15
Generation: 5 String: Ho a, allgnltios geHetiges Fitness: 17
Generation: 6 String: Holar alg nitgHs geHeticoi Fitness: 19
Generation: 7 String: Holar algoritl,s genetncos Fitness: 22
Generation: 8 String: Hola, algoritmos teteticos Fitness: 24
Generation: 9 String: Hola, algoritmos teteticos Fitness: 24
Generation: 10 String: Hola, algoritmos geneticos Fitness: 26

Resultado final: Hola, algoritmos geneticos | Fitness: 26 | Generación: 10
